In [73]:
import importlib

In [74]:
import gc
import sys
import time
sys.path.append("/home/shaow/jupyter_base/pycharm_001")
base = "/home/shaow/jupyter_base/pycharm_001"
import torch
import experiment.model.dglGCNModel as models
import graph_process as G
import obj.GraphBase as GB
import utils
import mysql_utils as mysql

In [104]:
importlib.reload(G)
appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge, \
           id_index_map ,index_id_map, index_keyword_map, index_ros_map, index_app_map , \
           app_dict = G.process_year_graph_simple(2019, process_null=False, process_old=False, filter=mysql.applyid_filter, quest='F01')



gb_2019 = GB.GraphBase(appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge,
           id_index_map, index_id_map, index_ros_map, index_app_map, index_keyword_map ,
           app_dict)

8.0.19
10
socket 10.0.82.237:3306
244800
apply id now
done init 1 nodes from apply id
done init 21168 nodes from application


In [105]:
print([gb_2019.iim[i] for i in gb_2019.appid_set])

['F01', 'F']


In [106]:
t_data = G.get_graph_edges(gb_2019)

(2, 230)
<class 'int'>
<class 'numpy.int64'>
(2, 740)
<class 'int'>
<class 'numpy.int64'>
(2, 263)
<class 'int'>
<class 'numpy.int64'>
(2, 5652)
<class 'int'>
<class 'numpy.int64'>
(2, 26986)
<class 'int'>
<class 'numpy.int64'>


In [107]:
# t_data = load_data('wn18')
# entities: 14951
# relations: 1345
# edges: 483142
# num_nodes = t_data.num_nodes
train_data = t_data.train
valid_data = t_data.validation
test_data = t_data.test
# num_rels = t_data.num_rels
num_nodes = gb_2019.node_count()
num_rels = gb_2019.rel_count()

In [109]:

print(test_data.shape)
print(train_data.shape)
print(valid_data.shape)
print(num_nodes)
print(num_rels)
for i in train_data[:,0] :
    if i == 21170 : print(1)

(3347, 3)
(27159, 3)
(3365, 3)
21169
5


In [110]:
# test_graph, test_node_id, test_rel, test_norm = G.make_test_graph(test_data, num_rels)
# test_node_id = torch.from_numpy(test_node_id)
# test_norm = torch.from_numpy(test_norm)
# # test_rel = torch.from_numpy(test_rel)
# test_edge_norm = node_norm_to_edge_norm(test_graph, test_norm.view(-1, 1))
test_graph, test_rel, test_norm = utils.build_test_graph(
        num_nodes, num_rels, train_data)
test_deg = test_graph.in_degrees(
            range(test_graph.number_of_nodes())).float().view(-1,1)
test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
test_rel = torch.from_numpy(test_rel)
test_norm = models.node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))

Test graph:


/home/shaow/jupyter_base/pycharm_001/utils.py:130: RuntimeWarning: divide by zero encountered in true_divide
  norm = 1.0 / in_deg


In [111]:
test_data = torch.from_numpy(test_data)
use_cuda = torch.cuda.is_available()

In [112]:
if use_cuda:
    torch.cuda.set_device('cuda:0')

In [113]:
model = models.LinkPredict(num_nodes,
                        256,
                        num_rels,
                    #="number of weight blocks for each relation"
                        num_bases=100,
                    # number of propagation rounds
                    # 3 because the trans road
                        num_hidden_layers=3,
                        dropout=0.2,
                        use_cuda=use_cuda,
                    # regularization weight
                        reg_param=0.01)

In [114]:
# test_graph, test_rel, test_norm = utils.build_test_graph(
#         num_nodes, num_rels, train_data)
# test_deg = test_graph.in_degrees(
#             range(test_graph.number_of_nodes())).float().view(-1,1)
# test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
# test_rel = torch.from_numpy(test_rel)
# test_norm = node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))
# print(test_node_id)

In [115]:
if use_cuda:
    model.cuda()

# build adj list and calculate degrees for sampling
adj_list, degrees = utils.get_adj_and_degrees(num_nodes, train_data)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model_state_file = 'model_state.pth'
forward_time = []
backward_time = []

# training loop
print("start training...")

epoch = 0
best_mrr = 0

start training...


In [116]:

while True:
    model.train()
    epoch += 1
    # perform edge neighborhood sampling to generate training graph and data
    g, node_id, edge_type, node_norm, data, labels = \
        utils.generate_sampled_graph_and_labels(
            train_data, sample_size=10000, split_size=0.5,
            num_rels=num_rels, adj_list=adj_list, degrees=degrees, negative_rate=5,
            sampler='uniform')
    # print("Done edge sampling")
    # set node/edge feature
    node_id = torch.from_numpy(node_id).view(-1, 1).long()
    edge_type = torch.from_numpy(edge_type)
    edge_norm = models.node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1))
    data, labels = torch.from_numpy(data), torch.from_numpy(labels)
    deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1)
    if use_cuda:
        node_id, deg = node_id.cuda(), deg.cuda()
        edge_type, edge_norm = edge_type.cuda(), edge_norm.cuda()
        data, labels = data.cuda(), labels.cuda()

    t0 = time.time()
    embed = model(g, node_id, edge_type, edge_norm)
    print(embed.shape)
    loss = model.get_loss(g, embed, data, labels)
    t1 = time.time()
    loss.backward()
    # grade norm = 1.0
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients
    optimizer.step()
    t2 = time.time()

    forward_time.append(t1 - t0)
    backward_time.append(t2 - t1)
    if epoch % 25 == 0:
        print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f} | Forward {:.4f}s | Backward {:.4f}s".
          format(epoch, loss.item(), best_mrr, forward_time[-1], backward_time[-1]))

    optimizer.zero_grad()

    # validation
    if epoch % 100 == 0:
        # gc.collect()
        # perform validation on CPU because full graph is too large
        if use_cuda:
            model.cpu()
        model.eval()
        print("start eval")
        print(test_graph)
        # if use_cuda:
        #     test_graph = test_graph.to('cuda:0')
        #     test_node_id, test_rel, test_norm = test_node_id.cuda(), test_rel.cuda(), test_norm.cuda()
        # embed = model.embedding.embedding
        # embed = embed(test_node_id)
        embed = model(test_graph, test_node_id, test_rel, test_norm)
        # torch.cuda.empty_cache()
        # gc.collect()
        # args.eval_batch_size = 500
        mrr = utils.calc_mrr(embed, model.w_relation, test_data,
                             hits=[1, 3, 10], eval_bz=500)
        # save best model
        if mrr < best_mrr:
            # min train epoch = 6000
            if epoch >= 6000:
                break
        else:
            best_mrr = mrr
            torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
                       model_state_file)
        if use_cuda:
            model.cuda()

torch.Size([11109, 256])
torch.Size([11089, 256])
torch.Size([11022, 256])
torch.Size([11162, 256])
torch.Size([11091, 256])
torch.Size([11037, 256])
torch.Size([11066, 256])
torch.Size([11080, 256])
torch.Size([11121, 256])
torch.Size([11080, 256])
torch.Size([11041, 256])
torch.Size([11101, 256])
torch.Size([11076, 256])
torch.Size([11032, 256])
torch.Size([11067, 256])
torch.Size([11078, 256])
torch.Size([11058, 256])
torch.Size([11054, 256])
torch.Size([11108, 256])
torch.Size([11056, 256])
torch.Size([11032, 256])
torch.Size([11104, 256])
torch.Size([11079, 256])
torch.Size([11148, 256])
torch.Size([11036, 256])
Epoch 0025 | Loss 0.4379 | Best MRR 0.0000 | Forward 0.0298s | Backward 0.0795s
torch.Size([11076, 256])
torch.Size([11036, 256])
torch.Size([11047, 256])
torch.Size([11116, 256])
torch.Size([11101, 256])
torch.Size([11098, 256])
torch.Size([11066, 256])
torch.Size([11071, 256])
torch.Size([11128, 256])
torch.Size([11068, 256])
torch.Size([11039, 256])
torch.Size([11110, 2